# MSA
https://github.com/vjhaupt/MSA_PDB_renumber/blob/master/msaPDBrenumber.py

## Make dictionary of all the sequences and the numbers of the aligned one
object define protein of interest <- create a class 
class Target
.get_canonical_residue_number() <- canonical numbering is the msa residue
(get residue number from the residue of the sequence)
(get residue in sequence and know the numbering)

## Use Mafft to align the sequences and read in the alignment to notebook
#### TODO: Mafft may not be the best to use since it only takes in one chain, so if mroe than one chain will be an issue: 
Would like to consider: https://academic.oup.com/nar/article/37/11/e83/1095196

Google: alignment multiple chains

In [1]:
from Bio import AlignIO
from pathlib import Path
import subprocess
from Bio import SeqIO
import os
import re

In [2]:
# https://github.com/choderalab/asapdiscovery/blob/seq-alignment/asapdiscovery-seqalign/align_code.ipynb
def multi_seq_alignment(input_file, output_file):
    # Run MAFFT in command line
    cmd = f"mafft {input_file} > {output_file}"
    subprocess.run(cmd, shell=True)

    # Read in the alignment to 
    align = AlignIO.read(output_file, "fasta")
    return align, output_file


In [7]:
local_path = Path('/home/pengs/fold_zika/fold_ns3')
input_file = local_path/"refseq_to_colabfold/sequence_for_msa.txt"
align, out_file = multi_seq_alignment(input_file, local_path/"msa.fasta")

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: -1 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
    1 / 7
done.

Constructing a UPGMA tree (efffree=0) ... 
    0 / 7
done.

Progressive alignment 1/2... 
STEP     6 / 6 
done.

Making a distance matrix from msa.. 
    0 / 7
done.

Constructing a UPGMA tree (efffree=1) ... 
    0 / 7
done.

Progressive alignment 2/2... 
STEP     6 / 6 
done.

disttbfast (aa) Version 7.525
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.


## The output of mafft

In [4]:
print('Alignment length')
print(align.get_alignment_length())
print('')
print('The alignment records')
for rec in align:
    print(rec.id + ' ' + rec.seq)


Alignment length
156

The alignment records
zikv_ns2b3 GETTDGVYRVMTRRLLGSTQVGVGVMQEGVFHTMWHVTKGAALRSGEGRLDPYWGDVKQDLVSYCGPWKLDAAWDGLSEVQLLAVPPGERAKNIQTLPGIFKTKDGDIGAVALDYPAGTSGSPILDKCGRVIGLYGNGVVIKNGSYVSAITQGKRE
ref|NP_776018.1| GDTTTGVYRIMTRGLLGSYQAGAGVMVEGVFHTLWHTTKGAALMSGEGRLDPYWGSVKEDRLCYGGPWKLQHKWNGHDEVQMIVVEPGKNVKNVQTKPGVFKTPEGEIGAVTLDYPTGTSGSPIVDKNGDVIGLYGNGVIMPNGSYISAIVQGER-
ref|YP_001527884.1| GDTTTGVYRIMTRGLLGSYQAGAGVMVEGVFHTLWHTTKGAALMSGEGRLDPYWGSVKEDRLCYGGPWKLQHKWNGQDEVQMIVVEPGKNVKNVQTKPGVFKTPEGEIGAVTLDFPTGTSGSPIVDKNGDVIGLYGNGVIMPNGSYISAIVQGER-
ref|NP_740321.1| ---SEGVYRIMQRGLFGKTQVGVGIHMEGVFHTMWHVTRGSVICHETGRLEPSWADVRNDMISYGGGWRLGDKWDKEEDVQVLAIEPGKNPKHVQTKPGLFKTLTGEIGAVTLDFKPGTSGSPIINRKGKVIGLYGNGVVTKSGDYVSAITQAER-
ref|NP_722463.1| ----DGIYRILQRGLLGRSQVGVGVFQEGVFHTMWHVTRGAVLMYQGKRLEPSWASVKKDLISYGGGWRFQGSWNAGEEVQVIAVEPGKNPKNVQTAPGTFKTPEGEVGAIALDFKPGTSGSPIVNREGKIVGLYGNGVVTTSGTYVSAIAQAK--
ref|NP_739587.2| AELEDGAYRIKQKGILGYSQIGAGVYKEGTFHTMWHVTRGAVLMHKGKRIEPSWADVKKDLISYGGGWKLEG

## Number each residue of the sequence
https://stackoverflow.com/questions/5983689/renumber-residues-in-a-protein-structure-file-pdb

number each residue within the input file for the colabfold (which will be more of what is in the colabfold)

number each residue within the aligned file to have new number of residue

## Read in the alignment file and renumber based on the first sequence (which I am assuming is being used as reference)

In [5]:
# Shove in the alignIO sequences from mafft and get the new number for each amino acid in the sequence
def get_new_renumber(align):
    res_num = dict()
    
    # Reference is the first sequence
    ref = align[0]
    # Initialize annotation for reference sequence, assume first residue is #1
    ref.letter_annotations["resnum"] = range(1,len(ref)+1)
    # Save the residue number
    res_num[ref.id] = list(ref.letter_annotations["resnum"])
    
    # Get a list of all other sequences to map back to the reference
    to_renumber = align[1:]
    for rec in to_renumber:
        # Initiate a new annotation for the sequences
        rec.letter_annotations["resnum"]=[None]*len(rec)
        # Set new residue numbers in rec based on alignment
        reslist = [[i,ref.letter_annotations["resnum"][i]] for i in range(len(ref)) if rec[i] != '-']
        for [i,r] in reslist:
            rec.letter_annotations["resnum"][i]=r
        # Set new residue numbers in the structure
        newresnums = [i for i in rec.letter_annotations["resnum"][:] if i != None]
        # Save to dict
        res_num[rec.id] = newresnums
    # Return the residue numbers of the aligned files
    # Dictionary with sequence id as the key and the residue numbers in a list as the value
    return res_num
    

In [6]:
align_num = get_new_renumber(align)

## Read from sequence that will be put into colabfold

In [8]:
fasta_sequences = list(SeqIO.parse(open(input_file),'fasta'))

In [15]:
# Current residue numbers
current_num = dict()
current_seq = dict()

for fasta in fasta_sequences:
    current_num[fasta.id] = list(range(1,len(fasta)+1))
    current_seq[fasta.id] = fasta.seq

In [17]:
current_seq

{'zikv_ns2b3': Seq('GETTDGVYRVMTRRLLGSTQVGVGVMQEGVFHTMWHVTKGAALRSGEGRLDPYW...KRE'),
 'ref|NP_776018.1|': Seq('GDTTTGVYRIMTRGLLGSYQAGAGVMVEGVFHTLWHTTKGAALMSGEGRLDPYW...GER'),
 'ref|YP_001527884.1|': Seq('GDTTTGVYRIMTRGLLGSYQAGAGVMVEGVFHTLWHTTKGAALMSGEGRLDPYW...GER'),
 'ref|NP_740321.1|': Seq('SEGVYRIMQRGLFGKTQVGVGIHMEGVFHTMWHVTRGSVICHETGRLEPSWADV...AER'),
 'ref|NP_722463.1|': Seq('DGIYRILQRGLLGRSQVGVGVFQEGVFHTMWHVTRGAVLMYQGKRLEPSWASVK...QAK'),
 'ref|NP_739587.2|': Seq('AELEDGAYRIKQKGILGYSQIGAGVYKEGTFHTMWHVTRGAVLMHKGKRIEPSW...TEK'),
 'ref|YP_001531172.2|': Seq('AELEEGVYRIKQQGIFGKTQVGVGVQKEGVFHTMWHVTRGAVLTHNGKRLEPNW...NAE')}

## Make single letter code into three letter code
https://stackoverflow.com/questions/72598009/convert-single-letter-code-to-3-letter-code-along-with-the-chain-numbering

### Residue ID in Prolif looks like: ResidueId(ASP, 110, A)
(three letter code, residue number, chain)

In [10]:
# Check what format I would need to map it to
#import prolif as plf
#fp = plf.Fingerprint.from_pickle('/home/pengs/fold_zika/combined/NP_722463.1.pkl')
#fp.ifp

In [11]:
d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
    'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}
d_flip = {v:k for k,v in d.items()}

# Input: 
# seq = sequence
# dic = amino acid dictionary that maps the three letter code to one letter code
# seq_num = the residue number in just the sequence
# chain = the chain the residue is on
# align_num = the residue number asigned to the residue from the mafft msa renumbering (to the canonical number) 
def triple_aa_renumber(seq, dic, seq_list, chain, align_list):   
    res = dict()
    for idx, char in enumerate(seq):
        code = dic.get(char, 'XXX')
        res[(code,seq_list[idx],chain)] = align_list[idx]
    return res

# change the triple amino acid to single amino acid
def trip_to_single_aa(trip_aa):
    single_aa = d[trip_aa]
    return single_aa

## Make great big dictionary with the sequence residue and the number
### map of the current sequence residue mapped to the MSA alignment number

In [12]:
map = dict()
for id in current_num:
    map[id] = triple_aa_renumber(current_seq[id], d_flip, current_num[id], 'A', align_num[id])

In [13]:
map['zikv_ns2b3']

{('GLY', 1, 'A'): 1,
 ('GLU', 2, 'A'): 2,
 ('THR', 3, 'A'): 3,
 ('THR', 4, 'A'): 4,
 ('ASP', 5, 'A'): 5,
 ('GLY', 6, 'A'): 6,
 ('VAL', 7, 'A'): 7,
 ('TYR', 8, 'A'): 8,
 ('ARG', 9, 'A'): 9,
 ('VAL', 10, 'A'): 10,
 ('MET', 11, 'A'): 11,
 ('THR', 12, 'A'): 12,
 ('ARG', 13, 'A'): 13,
 ('ARG', 14, 'A'): 14,
 ('LEU', 15, 'A'): 15,
 ('LEU', 16, 'A'): 16,
 ('GLY', 17, 'A'): 17,
 ('SER', 18, 'A'): 18,
 ('THR', 19, 'A'): 19,
 ('GLN', 20, 'A'): 20,
 ('VAL', 21, 'A'): 21,
 ('GLY', 22, 'A'): 22,
 ('VAL', 23, 'A'): 23,
 ('GLY', 24, 'A'): 24,
 ('VAL', 25, 'A'): 25,
 ('MET', 26, 'A'): 26,
 ('GLN', 27, 'A'): 27,
 ('GLU', 28, 'A'): 28,
 ('GLY', 29, 'A'): 29,
 ('VAL', 30, 'A'): 30,
 ('PHE', 31, 'A'): 31,
 ('HIS', 32, 'A'): 32,
 ('THR', 33, 'A'): 33,
 ('MET', 34, 'A'): 34,
 ('TRP', 35, 'A'): 35,
 ('HIS', 36, 'A'): 36,
 ('VAL', 37, 'A'): 37,
 ('THR', 38, 'A'): 38,
 ('LYS', 39, 'A'): 39,
 ('GLY', 40, 'A'): 40,
 ('ALA', 41, 'A'): 41,
 ('ALA', 42, 'A'): 42,
 ('LEU', 43, 'A'): 43,
 ('ARG', 44, 'A'): 44,
 ('SER

### TODO: Will need to note which chain it is on (also would like to map that to the chain on the original zikv ns2b3 structure)

## Find out where the active residues of each folded protein are based on the alignment
### Each active residue is likely based off of the ligand so would have different residues interacting for each of them
## Would need to know the active site to do the redocking of the ligand into the colabfold structures (asapdiscovery docking)

## Identify active residues for ligand bound to zika ns2b3
https://github.com/choderalab/asapdiscovery/blob/main/asapdiscovery-data/asapdiscovery/data/backend/plip.py

In [1]:
from asapdiscovery.data.backend.plip import get_interactions_plip
from asapdiscovery.data.backend.openeye import (
    load_openeye_sdf,
    load_openeye_pdb
)

# Apparently after I have prepped the proteins, I lose the labeling of the chains. They all become A. Also has different interactions. 
## Somehow more interactions without prepping the structures

## Loop through the originally bound strctures (not prepped) and identify the residues it interacts with

In [35]:
from pymol import cmd
import os
import re
from pathlib import Path
from openeye import oechem

In [36]:
def loop_plips_ligpath(ligand_path, protein_file, color_method, target):
    # dictionary of plip fingerprints for each ligand given protein
    lig_plip = dict()
    
    # Openeye protein
    protein_orig = load_openeye_pdb(protein_file)
    
    # Load in different ligands
    # Openeye ligand
    lig_pattern = r'(lig_\d+)\.sdf'
    
    # Read in the fragments in the file
    for root, dirs, files in os.walk(ligand_path):
        for fragment in files:
            # Find the name of the ligand
            match = re.match(lig_pattern, fragment)
            # If it is not a sdf file, don't try to do this
            if match == None:
                continue
            lig_name = match.group(1)
            # Load in the fragment as OE object
            pose_orig = load_openeye_sdf(ligand_path/fragment)

            # Get plip
            try:
                plip = get_interactions_plip(protein_orig, pose_orig, color_method, target)
                # Save the ligand plip in the dictionary
                lig_plip[lig_name] = plip
            except KeyError:
                print(f'Encounter issue in {lig_name}')
                print('KeyError: probably because there are no bindingsite thing')
            
    return lig_plip

In [37]:
# Path where I stored basically everything 
path = Path('/home/pengs/fold_zika')

In [38]:
# Get the dictionary of plips
# Can get the active residues from this
# color method is defined somewhere (from the plip function compute_fint_score)
color_method = 'fitness'
# target: need to check documentation for how named yaml
target = "ZIKV-NS2B-NS3pro"
lig_plip = loop_plips_ligpath(path/'fragments/', path/'zikv_ns2b3.pdb', color_method, target)

FileNotFoundError: /home/pengs/fold_zika/zikv_ns2b3.pdb does not exist!

## I Know there are a couple ligands that does not have detectable interactions for fingerprinting
### I do not know how to handle this since some of them are not going to be able to be prepped if this overlapses with the the complexes that are not able to be prepped by the other methods


In [ ]:
def get_all_ligand_in_path(lig_pattern, ligand_path, type = 'file'):
    ligs = []
    for root, dirs, files in os.walk(ligand_path):
        if type == 'dir':
            for fragment in dirs:
                # Find the name of the ligand
                match = re.match(lig_pattern, fragment)
                if match == None:
                    print(f'No match for {fragment}')
                else:
                    lig_name = match.group(1)
                    ligs.append(lig_name)
        if type == 'file':
            for fragment in files:
                # Find the name of the ligand
                match = re.match(lig_pattern, fragment)
                if match == None:
                    print(fragment)
                else:
                    lig_name = match.group(1)
                    ligs.append(lig_name)
    return set(ligs)


In [ ]:
# Read in all ligand names from the fragments files to list
lig_pattern = r'(lig_\d+)\.sdf'
ligs_all = get_all_ligand_in_path(lig_pattern, path/'fragments/')


In [ ]:
# Read in all ligand names from the orig_prepped files to list
lig_prep_pattern = r'(lig_\d+)_protein'
ligs_prepped = get_all_ligand_in_path(lig_prep_pattern, path/'orig_prepped/', type='dir')

In [ ]:
# See which one is missing
list(set(ligs_all) - set(ligs_prepped))

## The ligands that were not able to get residues with interactions with get_interactions_plip (63, 92, 72, 11, 78, 85)are not the same as the ones that failed to prep, so would be able to prep with identifying active residues

### Get the active residues of ligand 48 and ligand 4

## The format to use "HIS:41: :A"
### Getting this error: Warning: Unable to parse and utilize specified site residue TRP:69::B to define site in _lig_4_LIG - skipping

In [ ]:
# Parse the different names in the plip identified to find the active residue
def find_du_oe_active_res(lig_res_plip):
    pattern = r'(\d+)_(\D+)(\d+)\.(\w)'
    match = re.match(pattern, lig_res_plip)
    if match:
        aa = match.group(2)
        num = match.group(3)
        chain = match.group(4)
        # format into ways for oe-active-site-residue TEXT
        oe_active_define = aa + ':' + num + '::' + chain
        return oe_active_define
    else:
        print('No match found')

In [ ]:
# lig_4
lig_4_oe_text = ''
for key in lig_plip['lig_4']:
    lig_4_oe_text += find_du_oe_active_res(key) + ' '
print(lig_4_oe_text)

In [3]:
# These are not oemol residue ids, these are just the pdb id 
protein_orig = load_openeye_pdb('/home/pengs/fold_zika/test_prep/zikv_ns3.pdb')
pose_orig = load_openeye_sdf('/home/pengs/fold_zika/lig_sdfs/lig_0.sdf')
color_method = 'fitness'
target = "ZIKV-NS2B-NS3pro"
get_interactions_plip(protein_orig, pose_orig, color_method, target)

2024-03-25 16:52:29,266 [INFO] [plipcmd.py:124] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.3.0
2024-03-25 16:52:29,267 [INFO] [plipcmd.py:125] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2024-03-25 16:52:29,268 [INFO] [plipcmd.py:126] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2024-03-25 16:52:29,268 [INFO] [plipcmd.py:49] plip.plipcmd: starting analysis of tmp_complex.pdb
2024-03-25 16:52:29,851 [INFO] [plipcmd.py:165] plip.plipcmd: finished analysis, find the result files in /scratch/lsftmp/4940534.tmpdir/tmpnsg8_a6x/


{'0_ASP129.B': {'lig_at_x': '-7.322',
  'lig_at_y': '5.371',
  'lig_at_z': '-19.964',
  'prot_at_x': '-5.453',
  'prot_at_y': '3.925',
  'prot_at_z': '-22.856',
  'type': 'hydrogen_bond',
  'color': '#ffffff'},
 '1_TYR161.B': {'lig_at_x': '-8.664',
  'lig_at_y': '4.845',
  'lig_at_z': '-17.588',
  'prot_at_x': '-6.980',
  'prot_at_y': '1.381',
  'prot_at_z': '-18.116',
  'type': 'pi_stack',
  'color': '#ff9e83'},
 '2_TYR161.B': {'lig_at_x': '-6.655',
  'lig_at_y': '5.513',
  'lig_at_z': '-18.740',
  'prot_at_x': '-6.980',
  'prot_at_y': '1.381',
  'prot_at_z': '-18.116',
  'type': 'pi_stack',
  'color': '#ff9e83'}}

In [19]:
# Test out if this will work
protein_orig = load_openeye_pdb('/home/pengs/fold_zika/test_prep/zikv_ns3.pdb')
pose_orig = load_openeye_sdf('/home/pengs/fold_zika/lig_sdfs/lig_90.sdf')
color_method = 'fitness'
target = "ZIKV-NS2B-NS3pro"
get_interactions_plip(protein_orig, pose_orig, color_method, target)


2024-03-27 15:00:22,400 [INFO] [plipcmd.py:124] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.3.0
2024-03-27 15:00:22,401 [INFO] [plipcmd.py:125] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2024-03-27 15:00:22,402 [INFO] [plipcmd.py:126] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2024-03-27 15:00:22,402 [INFO] [plipcmd.py:49] plip.plipcmd: starting analysis of tmp_complex.pdb
2024-03-27 15:00:22,767 [INFO] [plipcmd.py:165] plip.plipcmd: finished analysis, find the result files in /scratch/lsftmp/5036205.tmpdir/tmpvvcldmw7/


{'0_GLN74.B': {'lig_at_x': '-15.857',
  'lig_at_y': '-7.892',
  'lig_at_z': '-7.970',
  'prot_at_x': '-16.114',
  'prot_at_y': '-5.603',
  'prot_at_z': '-4.976',
  'type': 'hydrophobic_interaction',
  'color': '#ffffff'},
 '1_ILE123.B': {'lig_at_x': '-13.125',
  'lig_at_y': '-11.441',
  'lig_at_z': '-9.032',
  'prot_at_x': '-10.643',
  'prot_at_y': '-9.909',
  'prot_at_z': '-11.120',
  'type': 'hydrophobic_interaction',
  'color': '#ffffff'},
 '2_THR118.B': {'lig_at_x': '-9.904',
  'lig_at_y': '-9.975',
  'lig_at_z': '-7.822',
  'prot_at_x': '-7.278',
  'prot_at_y': '-12.957',
  'prot_at_z': '-8.512',
  'type': 'hydrogen_bond',
  'color': '#ff7454'},
 '3_THR118.B': {'lig_at_x': '-9.904',
  'lig_at_y': '-9.975',
  'lig_at_z': '-7.822',
  'prot_at_x': '-7.278',
  'prot_at_y': '-12.957',
  'prot_at_z': '-8.512',
  'type': 'hydrogen_bond',
  'color': '#ff7454'}}

### Command to run
### Still unable to prep with this 
##### Would only take one argument for --oe-active-site-residue because just need one residue to find the pocket
asap-prep protein-prep --target ZIKV-NS2B-NS3pro --pdb-file orig_bind/lig_4_protein.pdb --output-dir orig_prepped --align zikv_ns2b3.pdb --use-dask --oe-active-site-residue TRP:69:B --ref-chain B --active-site-chain B


In [ ]:
# lig_48
lig_48_oe_text = ''
for key in lig_plip['lig_48']:
    lig_48_oe_text += find_du_oe_active_res(key) + ' '
print(lig_48_oe_text)

## Error: currently not resolving since would probably need to do redocking for generated structure not x-ray screen structure
ValueError: Failed to make design unit.
Warning: Truncating title to 80 characters for ASP-MET-TYR-ILE-GLU-ARG-ALA-GLY-ASP-ILE-THR-TRP-GLU-LYS-ASP-ALA-GLU-VAL-THR-GLY-ASN-SER-PRO-ARG-LEU-ASP-VAL-ALA-LEU-ASP-GLU-SER-GLY-ASP-PHE-SER-LEU-VAL-GLU
Warning: OEPlaceHydrogens: mol _lig_48_LIG has heavy atoms ' O   LYS 169   B 1  ' and 'C LIG 172     0  ' too close 0.34 < 0.5A
Warning: Detected residue with blank chain ID LIG 172     0


## Find all the different plip interactions
### If unable to get plip from unprepped zikv ns2b3 and ligand, try to get it in prepped zikv ns2b3 and ligand

In [56]:
lig_have_plip = list(lig_plip.keys())
list(set(ligs_all) - set(lig_have_plip))

[]

In [57]:
path = Path('/home/pengs/fold_zika/orig_prepped/lig_69_protein-0e036e6aa2f9dd85ee21c6ccac0d8516f559f4146cef6158afdfaca04b5046f5+BTRAZOCUAYTGAU-RPATYJTLNA-G')

# Openeye protein
protein_orig = load_openeye_pdb(path/'lig_69_protein.pdb')
# Openeye ligand
pose_orig = load_openeye_sdf(path/'lig_69_protein_ligand.sdf')
# color method is defined somewhere (from the plip function compute_fint_score)
color_method = 'fitness'
# target: need to check documentation for how named yaml
target = "ZIKV-NS2B-NS3pro"

# Example: intn_dict = get_interactions_plip(protein, pose, "fitness", target)
prep_69_fit = get_interactions_plip(protein_orig, pose_orig, color_method, target)

2024-03-21 14:48:36,652 [INFO] [plipcmd.py:124] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.3.0
2024-03-21 14:48:36,653 [INFO] [plipcmd.py:125] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2024-03-21 14:48:36,653 [INFO] [plipcmd.py:126] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2024-03-21 14:48:36,654 [INFO] [plipcmd.py:49] plip.plipcmd: starting analysis of tmp_complex.pdb
2024-03-21 14:48:38,705 [INFO] [plipcmd.py:165] plip.plipcmd: finished analysis, find the result files in /scratch/lsftmp/4832143.tmpdir/tmpc0gf0ml5/


## This list is surprisingly long, would like to know why
prep_69_fit

In [59]:
# Add this to the dictionary with all the other plips
lig_plip['lig_69'] = prep_69_fit

## Identify active residues by mapping to msa and then trying get prepped structures by identifying those as the active residues

In [ ]:
# The map of residues in structure to the sequence alignment spot
map['ref|zika_ns2b3|']
#map_flip = {v:k for k,v in map.items()}

## Find OEMol object residue number

In [19]:
from Bio import SeqIO
## How to get the type of residue and the number assigned to it from Openeye object
path = Path('/home/pengs/fold_zika/orig_prepped/lig_69_protein-0e036e6aa2f9dd85ee21c6ccac0d8516f559f4146cef6158afdfaca04b5046f5+BTRAZOCUAYTGAU-RPATYJTLNA-G')
#### Openeye protein
protein_orig = load_openeye_pdb(path/'lig_69_protein.pdb')

In [39]:
#### Iterate over the residues and print some information
def find_oe_res_id(protein):
    #### Get the residues from the molecule
    residues = oechem.OEGetResidues(protein)
    # Getting the residue ids in the protein file
    oe_id = dict()
    count = 1
    for residue in residues:
        #oe_thing[(residue.GetName(),residue.GetResidueNumber(),residue.GetExtChainID())] = count
        # Since did not get the chain ID before, would just go with A now
        oe_id[count] = (residue.GetName(),residue.GetResidueNumber(),'A')
        count += 1
    return oe_id

## Map the openeye sequence id to the (sequence residue id, canonical numbering)

In [40]:
def map_oe_res_to_seq(prot_file, protein_key, map):
    #### Openeye protein
    protein = load_openeye_pdb(prot_file)
    oe_id = find_oe_res_id(protein)
    # Find the starting sequence in the way oegraphmol loaded the object
    # Make a sequence out of the oemol residues 
    oe_start_seq = ''
    residues = oechem.OEGetResidues(protein)
    count = 1
    for residue in residues:
        # Just get the first 15 residues
        if count > 15:
            break
        oe_start_seq += trip_to_single_aa(residue.GetName())
        count += 1
    
    # Need to add 1 to match to the dict since this starts numbering at 0 with string while that one starts with 1
    oe_start = current_seq[protein_key].find(oe_start_seq) + 1
    # This is the position in the sequence that maps to the oemol starting position 
    
    # Find the first residue that matches the first residue in the pdb file
    matching_start = [(key, value) for key, value in map[protein_key].items() if key[1] == oe_start]
    match = matching_start[0][0][1]
    
    # Dictionary with key as oegraphmol id and value of (sequence residue id, canonical numbering)
    oe_mol_map = dict()
    for i in oe_id:
        # The residue id of the oegraphmol object
        oe_res = oe_id[i]
        # The mapping of this residue to the sequence mapping and canonical number
        map_match = [(key, value) for key, value in map[protein_key].items() if key[1] == match]
        # Save these in dict and go to next residue in mapped sequence
        oe_mol_map[oe_res] = map_match[0]
        match += 1
    return oe_mol_map

## oemol residue mapped to msa id

In [41]:
# Make dictionary of dictionaries (each protein own dictionary to map oemol resident id to sequence map id)
oe_map_dict = dict()

# Get the original protein numbering
prot_file = "/home/pengs/fold_zika/template_dir/zikv.pdb"
oe_map_dict['ref|zika_ns2b3|'] = map_oe_res_to_seq(prot_file, 'ref|zika_ns2b3|',map)

protein_path = Path('/home/pengs/fold_zika/only_prot')
for root, dirs, files in os.walk(protein_path):
    for prot_file in files:
        name_match = re.match(r'(.+).pdb', prot_file)
        name = name_match.group(1)
        key = f'ref|{name}|'
        oe_map_dict[key] = map_oe_res_to_seq(protein_path/prot_file, key, map)


KeyError: 'ref|YP_001621843.1|'

In [42]:
oe_map_dict['ref|zika_ns2b3|']

{('ASP', 50, 'A'): (('ASP', 1, 'A'), 1),
 ('MET', 51, 'A'): (('MET', 2, 'A'), 2),
 ('TYR', 52, 'A'): (('TYR', 3, 'A'), 3),
 ('ILE', 53, 'A'): (('ILE', 4, 'A'), 4),
 ('GLU', 54, 'A'): (('GLU', 5, 'A'), 5),
 ('ARG', 55, 'A'): (('ARG', 6, 'A'), 6),
 ('ALA', 56, 'A'): (('ALA', 80, 'A'), 80),
 ('GLY', 57, 'A'): (('GLY', 8, 'A'), 8),
 ('ASP', 58, 'A'): (('ASP', 9, 'A'), 9),
 ('ILE', 59, 'A'): (('ILE', 10, 'A'), 10),
 ('THR', 60, 'A'): (('THR', 11, 'A'), 11),
 ('TRP', 61, 'A'): (('TRP', 12, 'A'), 12),
 ('GLU', 62, 'A'): (('GLU', 86, 'A'), 86),
 ('LYS', 63, 'A'): (('LYS', 14, 'A'), 14),
 ('ASP', 64, 'A'): (('ASP', 15, 'A'), 15),
 ('ALA', 65, 'A'): (('ALA', 16, 'A'), 16),
 ('GLU', 66, 'A'): (('GLU', 17, 'A'), 17),
 ('VAL', 67, 'A'): (('VAL', 18, 'A'), 18),
 ('THR', 68, 'A'): (('THR', 19, 'A'), 19),
 ('GLY', 69, 'A'): (('GLY', 20, 'A'), 20),
 ('ASN', 70, 'A'): (('ASN', 21, 'A'), 21),
 ('SER', 71, 'A'): (('SER', 22, 'A'), 22),
 ('PRO', 72, 'A'): (('PRO', 23, 'A'), 23),
 ('ARG', 73, 'A'): (('ARG',

In [43]:
# Flip this so the canonical number matches the oemol numbering
canon_to_oe_map = dict()
for prot in oe_map_dict:
    original_dict = oe_map_dict[prot]
    canon_to_oe_map[prot] = {value[1]: key for key, value in original_dict.items()}

In [24]:
def find_map_active_res(lig_res_plip):
    pattern = r'(\d+)_(\D+)(\d+)\.(\w)'
    match = re.match(pattern, lig_res_plip)
    if match:
        aa = match.group(2)
        num = int(match.group(3))
        chain = match.group(4)
        # format of storing into mapped residue identifier
        id = (aa,num,'A')
        return id
    else:
        print('No match found')

## Find canonical number of residues making interaction with the ligand

In [46]:
# Make a dictionary with the key: ligand and value: list of canonical number of protein residue that interacts with ligand in zikv ns2b3
orig_res_num = dict()

for lig in lig_plip:
    lig_res = list(lig_plip[lig].keys())
    # make a list of canonical numbers of residues that interact with said ligand in the original form
    reses = []
    canon_res_nums = []
    
    for res in lig_res:
        # map openeye residue id to 
        res_id = find_map_active_res(res)
        map_id = oe_map_dict['ref|zika_ns2b3|'][res_id][0]
        
        # Now find the position of that on the map
        #map_id = (map_id[0],map_id[1],'B')
        canonical_num = map['ref|zika_ns2b3|'][map_id]
        canon_res_nums.append(canonical_num)
    # Store this under the ligand
    orig_res_num[lig] = list(set(canon_res_nums))

In [ ]:
orig_res_num['lig_8']

In [25]:
# Reverse the dictionary and get residue in each protein from the canonical number
map_flip = dict()
for prot in map:
    map_flip[prot] = {v:k for k,v in map[prot].items()}

In [26]:
# Make residue format for prepping residue
def find_active_res_from_msa(protein_key, canon_num):
    res = map_flip[protein_key][canon_num]
    oe_active_define = res[0] + ':' + str(res[1]) + '::' + res[2]
    return oe_active_define

In [ ]:
# Trying with the cannon number 50 to see if this works
canon_num = 50
# key: the key name of protein
for key in map_flip:
    print(key)
    print(find_active_res_from_msa(key, canon_num))

In [ ]:
# Find the residues in the sequence that maps 

## This function takes in OE formatted string of active site residue to use

In [27]:
# Find the oemol number of the protein from the canonical number in the msa
def find_oemol_from_canon(protein, canon_nums,canon_to_oe_map):
    # protein: key name the protein that the ligand is having interaction with
    # canon_nums: list of canonical numbering of residues that are in active site
    # Return: a list of the active residues in format able to put into asap-prep module (OE formatted string of active site residue)
    act_res_list = []
    for canon in canon_nums:
        # check if this canonical number matches a residue instead of a blanks
        if canon in canon_to_oe_map[protein]:
            res = canon_to_oe_map[protein][canon]
            oe_active_define = res[0] + ':' + str(res[1]) + '::' + res[2]
            act_res_list.append(oe_active_define)
    return act_res_list

In [28]:
find_oemol_from_canon('ref|NP_739587.2|', [39+154])

TypeError: find_oemol_from_canon() missing 1 required positional argument: 'canon_to_oe_map'

In [44]:
find_oemol_from_canon('ref|zika_ns2b3|', [39+154], canon_to_oe_map)

['LYS:169::A']

## These numbers are somehow also the residue id in pymol

## Generate the map of oemol residues from the canonical numbering
### Protein big key, ligand small key within

In [53]:
# Dictionary of 
oe_msa_active_map = dict()

# Find the matching ligand interaction most likely in the generated protein based on msa
for lig in lig_plip:
    #print(lig)
    canon_nums = orig_res_num[lig]
    oe_msa_active_map[lig] = dict()
    for prot in oe_map_dict:
        #print(prot)
        oe_msa_active_map[lig][prot] = find_oemol_from_canon(prot, canon_nums,canon_to_oe_map)

In [54]:
oe_msa_active_map

{'lig_35': {'ref|zika_ns2b3|': ['TYR:161::A'],
  'ref|NP_776018.1|': ['TYR:147::A'],
  'ref|YP_001527884.1|': ['TYR:147::A'],
  'ref|YP_001531172.2|': ['TYR:147::A'],
  'ref|NP_739587.2|': ['TYR:147::A'],
  'ref|NP_722463.1|': ['TYR:142::A'],
  'ref|NP_740321.1|': ['TYR:143::A'],
  'ref|YP_001621843.1|': ['TYR:169::A']},
 'lig_25': {'ref|zika_ns2b3|': ['TYR:161::A', 'ALA:132::A', 'SER:135::A'],
  'ref|NP_776018.1|': ['TYR:147::A', 'THR:118::A', 'SER:121::A'],
  'ref|YP_001527884.1|': ['TYR:147::A', 'THR:118::A', 'SER:121::A'],
  'ref|YP_001531172.2|': ['TYR:147::A', 'PRO:118::A', 'SER:121::A'],
  'ref|NP_739587.2|': ['TYR:147::A', 'PRO:118::A', 'SER:121::A'],
  'ref|NP_722463.1|': ['TYR:142::A', 'PRO:113::A', 'SER:116::A'],
  'ref|NP_740321.1|': ['TYR:143::A', 'PRO:114::A', 'SER:117::A'],
  'ref|YP_001621843.1|': ['TYR:169::A', 'PRO:140::A', 'SER:143::A']},
 'lig_19': {'ref|zika_ns2b3|': ['TYR:161::A', 'ASP:129::A', 'VAL:155::A'],
  'ref|NP_776018.1|': ['TYR:147::A', 'ASP:115::A', 'ILE

## Need to have prepped structure to dock
### so need to have all the structures prepped before re-docking
#### Have a way to prep structure by giving it the active residues that it interacts with and getting the prepped structure by strong-arming it (--oe_active_site_residue)
https://github.com/choderalab/asapdiscovery/blob/cf611d1ab9e9f44255c99df91992aa550ed35a95/asapdiscovery-modeling/asapdiscovery/modeling/cli.py

## Identify Structures that need to be prepped because did not with normal call

In [ ]:
# prep_path: the path with all the folders of prepared protein ligand pairs
# all_ligs: a list with all the available ligands
def find_unprep_ligs(prep_pattern,prep_path,all_ligs):
    # Find all the ligands that has been prepared already from the prep output directory
    lig_have_plip = get_all_ligand_in_path(prep_pattern, prep_path, type = 'dir')

    # Find the ones missing ligands by comparing the found ligands in path to a list of all ligands
    diff = list(set(all_ligs) - set(lig_have_plip))
    return diff

In [ ]:
# Read in all ligand names from the fragments files to list
lig_pattern = r'(lig_\d+)\.sdf'
ligs_all = get_all_ligand_in_path(lig_pattern, path/'fragments/')

# Test with NP_722463.1
prep_pattern = lig_prep_pattern = r'\D+_\d+.\d_(lig_\d+)'
find_unprep_ligs(prep_pattern, '/home/pengs/fold_zika/combined/NP_722463.1_prepped',ligs_all)


### Before I figure out a better way, I will be customizing the prepping command to each one based on the identified active residue and writing out the commands to a giant file to run again

In [ ]:
# Return a list of all the files
def traverse_files(directory):
    all_files = []
    # Traverse files within a directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            all_files.append(file)
    return all_files

In [ ]:
command = 'asap-prep protein-prep --target ZIKV-NS2B-NS3pro'
file_command = ' --pdb-file'
output_command = ' --output-dir'
align_command = ' --align'
folder = ' /home/pengs/fold_zika/combined/'
active_res_command = '--oe-active-site-residue'
append = ' --ref-chain B --active-site-chain B --use-dask'

In [ ]:
# Get all the different commands into a single file since somehow only works when submit individually
run_file = Path("/home/pengs/fold_zika/")
with open(run_file/'prep_files.sh','w') as f:
    f.write('#!/bin/bash\n')
    f.write('\n')
    for seq in best:
        directory_path = os.path.join(combine_directory, seq)
        path = ' ' + directory_path + '/'
        files = traverse_files(directory_path)
        for file in files:
            name = find_name_in_file(r'(.+)\.pdb', file)
            path = folder + seq + '/'
            # Save each prepped protein-ligand folder in a protein file outside current folder
            # But keep within total working directory
            output_path = folder + seq + '_prepped/'
            output_dir = name
            # command for running preparation
            for res in oe_msa_active_map[prot][lig]:
                cmd = command + file_command + path + file + output_command + output_path + output_dir
                cmd += align_command + path + file + append + active_res_command + active_res + '\n'
                f.write(cmd)

for seq in best:
    prepped_dir = seq + "_prepped"
    directory_path = os.path.join(combine_directory, prepped_dir)
    os.mkdir(directory_path)

## What to do instead of pursuing ligand interactions
1. Re-Dock
2. ligand overlap in space
   see if the poses are flipped to see if they are different
   easy to compare between ligand because
3. oe: shape overlay
   how much volume are overlaped
   https://docs.eyesopen.com/toolkits/python/oechemtk/OEChemConstants/OEPerceived.html#OEChem::OEPerceived

### I think the cross-docking pipeline automatically does some protein-prepping, I can not find any ways to tell it to reference the correct chain (should be B, but keep on referencing A and causing error); would like ot know if I need to prep it before docking or if I could alter the workflow for crossdocking in local environment and prep it within the docking workflow (or could do so beforehand)
### Now I somehow got the prepping to run, but not the docking? Even though I did not specify chain B (but for the folded structure and not original protein (did not try yet))

 File "/lila/home/pengs/asapdiscovery/asapdiscovery-docking/asapdiscovery/docking/cli.py", line 265, in cross_docking
    cross_docking_workflow(inputs)
  File "/lila/home/pengs/asapdiscovery/asapdiscovery-docking/asapdiscovery/docking/workflows/cross_docking.py", line 147, in cross_docking_workflow
    prepped_complexes = prepper.prep(
  File "/lila/home/pengs/asapdiscovery/asapdiscovery-modeling/asapdiscovery/modeling/protein_prep.py", line 157, in prep
    outputs = self._prep(inputs=inputs)
  File "/lila/home/pengs/asapdiscovery/asapdiscovery-modeling/asapdiscovery/modeling/protein_prep.py", line 310, in _prep
    raise ValueError("Failed to make design unit.")
ValueError: Failed to make design unit.

# Redock
asap-docking cross-docking --help

Usage: asap-docking cross-docking [OPTIONS]

  Run cross docking on a set of ligands, against a set of targets.

Options:
  --target [EV-D68-Capsid|SARS-CoV-2-N-protein|SARS-CoV-2-Mac1|EV-A71-3Cpro|ZIKV-NS2B-NS3pro|DENV-NS2B-NS3pro|EV-A71-Capsid|SARS-CoV-2-Mpro|MERS-CoV-Mpro|EV-D68-3Cpro]
                                  The target for the workflow  [required]
  --use-omega                     Whether to use OEOmega conformer enumeration
                                  before docking (slower, more accurate)
  --omega-dense                   Whether to use dense conformer enumeration
                                  with OEOmega (slower, more accurate)
  --allow-retries                 Whether to allow POSIT to retry with relaxed
                                  parameters if docking fails (slower, more
                                  likely to succeed)
  --allow-final-clash             Allow clashing poses in last stage of
                                  docking
  --multi-reference               Whether to pass multiple references to the
                                  docker for each ligand instead of just one
                                  at a time
  --structure-selector [MCSSelector|PairwiseSelector|LeaveOneOutSelector|LeaveSimilarOutSelector|SelfDockingSelector]
                                  The type of structure selector to use.
  -l, --ligands FILE              File containing ligands
  --pdb-file FILE                 Path to a pdb file containing a structure
  --fragalysis-dir DIRECTORY      Path to a directory containing fragments to
                                  dock.
  --structure-dir DIRECTORY       Path to a directory containing structures.
  --save-to-cache / --no-save-to-cache
                                  If the newly generated structures should be
                                  saved to the cache folder.
  --cache-dir DIRECTORY           Path to a directory where design units are
                                  cached.
  --use-only-cache                Whether to only use the cache.
  --use-dask                      Whether to use dask for parallelism.
  --dask-type [local|local-gpu|lilac-gpu|lilac-cpu]
                                  The type of dask cluster to use. Can be
                                  'local', 'lilac-cpu' or  'lilac-gpu'.
  --dask-failure-mode [raise|skip]
                                  The failure mode for dask. Can be 'raise' or
                                  'skip'.  [default: DaskFailureMode.SKIP]
  --output-dir DIRECTORY          The directory to output results to.
  --overwrite / --no-overwrite    Whether to overwrite the output directory if
                                  it exists.
  --input-json FILE               Path to a json file containing the inputs to
                                  the workflow,  WARNING: overrides all other
                                  inputs.
  --loglevel [DEBUG|INFO|WARNING|ERROR|CRITICAL]
                                  The log level to use.  [default: INFO]
  --walltime TEXT                 The walltime to use for the dask_jobqueue
                                  cluster (if used) in dask format, e.g. '1h'
  --help                          Show this message and exit.

## Redock the ligands
https://github.com/choderalab/asapdiscovery/blob/main/asapdiscovery-modeling/asapdiscovery/modeling/modeling.py#L175

### Somehow the asapdiscovery method of docking uses smile strings for the ligand input, would explore all poses, so may be good
## Probably do this
Use Cross-docking --> more underlying docking options from command line

https://www.notion.so/asapdiscovery/HOWTO-Running-the-small-scale-docking-workflow-V2-44560183c5a7472c943a5b13fbce96b3
https://www.notion.so/asapdiscovery/HOWTO-Running-the-large-scale-docking-workflow-V2-32c3603915764ae5a85027e41d267ead
https://github.com/choderalab/asapdiscovery/blob/main/asapdiscovery-docking/asapdiscovery/docking/docking.py

asap-prep small-scale 
--target ZIKV-NS2B-NS3pro
--use-dask
--pdb-file my_structure.pdb


In [ ]:
# may need to make the ligand a smi file from the sdf


In [2]:
from rdkit import Chem

In [3]:
# Test
sdf = Chem.SDMolSupplier( '/home/pengs/fold_zika/lig_sdfs/lig_0.sdf' )
with open('/home/pengs/fold_zika/test_dock/lig_0.smi', 'w') as f:
    for mol in sdf:
        smi = Chem.MolToSmiles(mol)
        f.write("{}\n".format(smi))

## POSIT
https://github.com/choderalab/asapdiscovery/blob/main/asapdiscovery-docking/asapdiscovery/docking/workflows/cross_docking.py
https://docs.eyesopen.com/applications/oedocking/theory/posit_theory.html

give strcuture without ligand in it, will not make design unit
could not decide where to put the ligand
so need to get active site residues

In [ ]:
# load from file
    logger.info(f"Loading ligands from file: {inputs.ligands}")
    molfile = MolFileFactory(filename=inputs.ligands)
    query_ligands = molfile.load()

# dock pairs
    logger.info("Running docking on selected pairs")
    docker = POSITDocker(
        relax=inputs.relax,
        posit_method=inputs.posit_method,
        use_omega=inputs.use_omega,
        omega_dense=inputs.omega_dense,
        num_poses=inputs.num_poses,
        allow_low_posit_prob=inputs.allow_low_posit_prob,
        low_posit_prob_thresh=inputs.low_posit_prob_thresh,
        allow_final_clash=inputs.allow_final_clash,
        allow_retries=inputs.allow_retries,
    )
    results = docker.dock(
        sets,
        output_dir=output_dir / "docking_results",
        use_dask=inputs.use_dask,
        dask_client=dask_client,
        dask_failure_mode=inputs.dask_failure_mode,
    )

    n_results = len(results)
    logger.info(f"Docked {n_results} pairs successfully")
    if n_results == 0:
        raise ValueError("No docking results generated, exiting")
    del pairs

## Autodock Vina
https://www.notion.so/asapdiscovery/look-into-AutoDock-Vina-288e8f9bed554ba78c173bec19ccd614
https://autodock-vina.readthedocs.io/en/latest/docking_basic.html 
https://autodock-vina.readthedocs.io/en/latest/docking_python.html
https://vina.scripps.edu/
https://iwatobipen.wordpress.com/2021/09/04/cross-docking-study-with-python-vina-pymol-rdkit/comment-page-1/
https://github.com/ccsb-scripps/AutoDock-Vina/blob/develop/build/python/vina/vina.py

#### Installation
https://autodock-vina.readthedocs.io/en/latest/docking_requirements.html
https://ccsb.scripps.edu/adfr/
https://qvina.github.io/

#### Pymol plugins
https://pymolwiki.org/index.php/Dockingpie
https://www.biostars.org/p/12803/

## Short MD to make things look more reasonable
https://www.notion.so/asapdiscovery/Frag-Faux-Foldalysis-short-relaxation-MD-on-protein-ligand-complexes-e07de289ffcd4aab80f68d666f701558

## After redocking
1. realign proteins
2. figure out if the ligand still occupy about the same space
   - might be determined by which residues (by matching are nearby)
   - might just be determined by the 3D space it is defined in (if the space occupied is the same as prior)

## Notes from AviDD 3/20/24 
- how did in silico screen
- known active site and enzymatic properties
- masking off rest of protein for screen
- some amino acids in some active site known
- masking known cysteine 